In [53]:
import re
import numpy as np
from tqdm import tqdm
import psycopg2
from contextlib import contextmanager
import pandas as pd
from requests import Timeout
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id='3b39684f47a04dac88139d83421c9bab',
    client_secret='aecfeb0c94654fb7a776a75d3d4ba597'))

In [2]:
@contextmanager
def connect():
    """Connect to the PostgreSQL database server"""
    # connect to the PostgreSQL server
    print('Connecting to the PostgreSQL database...')
    conn = psycopg2.connect(
        host='localhost',
        port='5433',
        database='musicbrainz_db',
        user="musicbrainz",
        password="musicbrainz")
    c = conn.cursor()
    try:
        yield c
    finally:
        c.close()

def iter_row(cursor, size=10):
    while True:
        rows = cursor.fetchmany(size)
        if not rows:
            break
        for row in rows:
            yield row

def retry(fn, max_num_trials):
    trials = 0
    while trials < max_num_trials:
        trials += 1
        try:
            return fn()
        except Timeout as e:
            if trials >= max_num_trials:
                raise e

In [ ]:
songs = []
title_pattern = re.compile('^[A-Za-z][A-Za-z ]*$')
with connect() as c:
    c.execute(
        "SELECT DISTINCT LOWER(recording.name), LOWER(artist.name) FROM recording "
        "   INNER JOIN artist_credit_name ON recording.artist_credit = artist_credit_name.artist_credit "
        "   INNER JOIN artist ON artist_credit_name.artist = artist.id "
    )

    while True:
        rows = c.fetchmany(100)
        if not rows:
            break
        songs.extend([row for row in rows if title_pattern.match(row[0]) and len(row[0]) > 1 and title_pattern.match(row[1]) and len(row[1]) > 1])

# Insight

- Song titles extremely ambiguous -> most titles appear under several artists
- Lots of rather unknown songs -> no cover available
- Lots of spam data -> requires tedious cleanup methods

In [ ]:
songs_df = pd.DataFrame(songs)
songs_df.columns = ['title', 'artist']
songs_df.to_csv('songs.csv', index=False)

In [38]:
songs_df = pd.read_csv('songs.csv')

Number of songs

In [39]:
len(songs_df)

11002480

Number of songs whose title appear more than once

In [40]:
v = songs_df.title.value_counts()
songs_df[songs_df.title.isin(v.index[v > 1])]

,title,artist
1,a a a,lildrughill
2,a a a,plk
3,a a a,rocket
12,a a e i o,b fachada
13,a a e i o,minta
...,...,...
11002473,zzzzzzzzzz,adam cadell
11002474,zzzzzzzzzz,brad fotsch
11002475,zzzzzzzzzz,carter thornton
11002476,zzzzzzzzzz,hannes


Artists who have at least 50 songs

In [50]:
v = songs_df.artist.value_counts()
artists = list(songs_df[songs_df.artist.isin(v.index[v >= 50])].groupby(['artist']).groups.keys())

44711

Get popularity for each artist using the spotify search API

In [ ]:
popularities = dict()

In [55]:
# It seems that one can only make 25000 requests per IP/user per day on the spotify search API.
for i, artist in tqdm(enumerate(artists)):
    try:
        results = sp.search(q=artist, limit=1, type='artist')
        items = results['artists']['items']
        if len(items) == 0:
            continue
        # Check whether the exact name of the artist is contained in the results!
        # Otherwise, a completely unknown artist might be interpreted as the abbreviation of a popular artist.
        # For example, "meg" is interpreted by "Meghan Trainor" but "meg" in our dataset corresponds to some
        # completely unknown artist.
        name = items[0]['name'].lower()
        if name != artist:
            continue
        popularities[artist] = items[0]['popularity']
    except spotipy.SpotifyException as e:
        print(f'Exception raised on artist "{artist}"')
        print(e)

15925it [25:34, 10.38it/s]


In [59]:
popularity_df = pd.DataFrame(popularities.items(), columns=['artist', 'popularity'])
popularity_df.to_csv('popularity.csv', index=False)

In [14]:
popularity_df = pd.read_csv('popularity.csv')

Grab only songs from popular artists

In [62]:
most_popular_artists_df = popularity_df[popularity_df['popularity'] >= 70]
most_popular_artists_df.to_csv('most_popular_artists.csv', index=False)
popular_songs_df = songs_df[songs_df['artist'].isin(most_popular_artists_df['artist'])]

In [64]:
popular_songs_df.to_csv('popular_songs.csv', index=False)